**SYPA: Fundamental Analysis of Foreign Direct Investment** <br>
*3_EDA* <br>
Harvard SYPA <br>
User: Jake Schneider <br>
Date Created: February 8, 2020 <br>
Date Updated: February 8, 2020

____

**Run R and Python in the same notebook** <br>
Docs: https://stackoverflow.com/questions/39008069/r-and-python-in-one-jupyter-notebook

----

**Load Packages**

In [2]:
#Import libraries
import sys
import pandas as pd
from datetime import date, datetime, time, timedelta
import pendulum
import json
import requests
import numpy as np
import math

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style='ticks', context='talk')

from matplotlib.offsetbox import AnchoredText
from sklearn import linear_model
from sklearn.metrics import r2_score
import statsmodels.api as sm
#import fbprophet

import warnings
import itertools

from flask import jsonify, make_response

from varname import varname

from matplotlib.backends.backend_pdf import PdfPages
from fpdf import FPDF
from PIL import Image, ImageDraw, ImageFont

In [3]:
# Create function 'jprint'

def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

----

**Load Data**

In [6]:
analysis_df = pd.read_csv('../../2_Inputs/Final/analysis_df.csv')
analysis_df = analysis_df.drop(["Unnamed: 0"], axis = 1)
analysis_df.head()

,country,date,code,iso2Code,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,...,Relative.wage.of.Professionals.in.private.sector..using.clerk.as.reference..y,Relative.wage.of.Senior.officials.in.private.sector..using.clerk.as.reference..y,Relative.wage.of.Technicians.in.private.sector...using.clerk.as.reference..y,Female.to.male.wage.ratio.in.the.public.sector..using.median..y,Female.to.male.wage.ratio.in.the.public.sector..using.mean..y,Relative.wage.of.Professionals.in.public.sector..using.clerk.as.reference..y,Relative.wage.of.Senior.officials.in.public.sector..using.clerk.as.reference..y,Relative.wage.of.Technicians.in.public.sector..using.clerk.as.reference..y,Wage.bill.as.a.percentage.of.GDP.y,Wage.bill.as.a.percentage.of.Public.Expenditure.y
0,Afghanistan,1960.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1961.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1962.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1963.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1964.0,AFG,AF,South Asia,South Asia,Low income,IDA,Kabul,69.1761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
